<a href="https://colab.research.google.com/github/chain-veerender/LLM-RAG-explained-clearly/blob/main/Json_poc_SLM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ========== SETUP ==========
import os
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
# Install dependencies
!pip install -q langchain langchain-community langchain_huggingface chromadb transformers accelerate jsonformer docx2txt safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 113.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.3 MB/s eta 

In [3]:
# ========== FILE UPLOAD ==========
from google.colab import files
uploaded = files.upload()  # Upload your .docx file



Saving dummy_1.docx to dummy_1.docx


In [4]:
# ========== DOCUMENT LOADING ==========
from langchain_community.document_loaders import Docx2txtLoader

doc_path = next(iter(uploaded))  # Load the uploaded file
loader = Docx2txtLoader(doc_path)
evidence_doc = loader.load()


In [5]:

# ========== TEXT SPLITTING ==========
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=0)
chunked_docs = splitter.split_documents(evidence_doc)


In [6]:

# ========== EMBEDDING & VECTOR STORE ==========
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Use a lightweight sentence transformer for embeddings
embeds = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(chunked_docs, embedding=embeds)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 7})



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
question = """
Create JSON from context with the following rules:
- Chemical substances are identified by the Substance name, Substance group, and CAS number and/or EC number.
- CAS Number format: X-Y-Z (X: 2–7 digits, Y: 2 digits, Z: 1 digit).
- EC Number format: X-Y-Z (X: 3 digits, Y: 3 digits, Z: 1 digit).
- Extract: Manufacturer Name, Document Name, Document Date, Revision Number, Components (list, series, manufacturer), Regulation type/version/exemption, Compliance result, Substances list, CAS, concentrations.
"""

In [8]:
# ========Get relevant documents from retriever===========
#relevant_docs = retriever.get_relevant_documents(question)
relevant_docs = retriever.invoke(question)

# Join their contents into a single string (for context)
rag_context = "\n\n".join(doc.page_content for doc in relevant_docs)

In [9]:
# ========== MODEL SETUP (CPU) ==========
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

#model_name = "tiiuae/falcon-rw-1b"  # Small model that fits in Colab CPU
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#model = AutoModelForCausalLM.from_pretrained(model_name)
#model.to("cpu")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")



tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

In [10]:
# ========== PROMPT TEMPLATE ==========
from langchain.prompts import PromptTemplate

prompt_template = """
<|system|>
Answer the question based only on evidence from the context. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>
"""
prompt_use = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


In [12]:
# ========== JSON SCHEMA (sometimes small models are hallucinating with this structure) ==========


coc = {
  "type": "object",
  "properties": {
    "coc": {
      "type": "object",
      "properties": {
        "Manufacturer Name": {"type": "string"},
        "Document Name": {"type": "string"},
        "Document Date": {"type": "string"},
        "Revision Number": {"type": "string"},
        "Components List": {
          "type": "array", "maxItems": 5, "items": {"type": "string"}
        },
        "Components Series": {
          "type": "array", "maxItems": 5, "items": {"type": "string"}
        },
        "Components Manufacturer": {
          "type": "array", "maxItems": 5, "items": {"type": "string"}
        },
        "Regulation type": {"type": "string"},
        "Regulation version": {"type": "number"},
        "Regulation exemption": {"type": "boolean"},
        "Compliance result": {"type": "boolean"},
        "Non compliant substances list": {
          "type": "array", "maxItems": 5, "items": {"type": "string"}
        },
        "CAS numbers": {
          "type": "array", "maxItems": 5, "items": {"type": "string"}
        },
        "concentration": {"type": "string"}
      }
    }
  }
}


In [11]:
#========== Json schema (Minimal to avoid large computations)========================

coc = {
  "type": "object",
  "properties": {
    "coc": {
      "type": "object",
      "properties": {
        "Manufacturer Name": {"type": "string"},
        "Document Name": {"type": "string"},
        "Document Date": {"type": "string"},
        "Compliance result": {"type": "boolean"},
        "Regulation type": {"type": "string"},
        "Non compliant substances list": {
          "type": "array",
          "maxItems": 2,
          "items": {"type": "string"}
        }
      }
    }
  }
}

In [17]:
# ========== CONTEXT + QUESTION ==========
sample_context =  """
April 4, 2022

To: Whom It May Concern

Re: California Proposition 65 Compliance - All Dummy2 & Cable Part Numbers
Our products may contain ingredients such as DIDP or antimony trioxide; however, because our products are infrequently handled it is exempt from labeling per a NEMA consent judgment (Section 7 of the Consent Judgment in Mateel Environmental Justice Foundation v. Sprint Communications et al., and Mateel Environmental Justice Foundation v. Belkin Components et al., San Francisco Superior Court Cases Nos. 312962 and 320342).

The February 25, 2022 CA Prop 65 substances can be found at:
https://oehha.ca.gov/proposition-65/proposition-65-list
The information provided in this Product Disclosure, and the identification of materials listed as
reportable or restricted within the Product Disclosure, is correct to the best of Dummy2’s knowledge,
information and belief at the date of its publication. The information provided in the Product
Disclosure is designed only as a general guide for the safe handling, storage, and any other operation
of the product itself or the one that it becomes a part of. This Product Disclosure is not to be
considered a warranty or quality specification. Regulatory information is for guidance purposes only.
Product users are responsible for determining the applicability of legislation and regulations based on
their individual usage of the product.

Authorized Signatory for the Dummy2:

Dummy Name

Director of Engineering
DUMMY2

1/4/22
"""

question = """
Create JSON from context with the following rules:
- Chemical substances are identified by the Substance name, Substance group, and CAS number and/or EC number.
- CAS Number format: X-Y-Z (X: 2–7 digits, Y: 2 digits, Z: 1 digit).
- EC Number format: X-Y-Z (X: 3 digits, Y: 3 digits, Z: 1 digit).
- Extract: Manufacturer Name, Document Name, Document Date, Revision Number, Components (list, series, manufacturer), Regulation type/version/exemption, Compliance result, Substances list, CAS, concentrations.
"""


In [21]:
sample_context2 =  """
Dummy 1
Product Compliance Certificate
Restriction of the use of certain Hazardous Substances - RoHS2 - 2011-65/EU
 The EC RoHS Recast Directive 2011/65/EU restricts the use of the hazardous substances listed below in electrical equipment
The maximum concentration values of the restricted substances by weight are:
Based on information provided by its suppliers dummy1 designates the product listed below as:
RoHS Compliant
For orders placed on the date of this certificate
RoHS Compliant - means that dummy1 supplier has confirmed that the substances are either not present or
only present at concentrations under the maximum
Not RoHS Compliant - means that dummy1 supplier has confirmed that the substances are present at
concentrations over the maximum limits and no exemptions apply.
RoHS Compliant by Exemption - means that dummy1 supplier confirmed that one or more substances are
present above limit but the usage is permitted by the exemptions named below.
RoHS Not Applicable - means that dummy1 supplier has said that EU RoHS does not apply for this part.
RoHS Data Not Available - means that either dummy1 supplier has not yet checked for the substances or that
the data is under review.
RoHS 2 Amendment 2015/863
RoHS 2 has been modified by amendment 2015/863. This amendment will be in force for most products from
22nd July 2019 and restricts the use of four additional hazardous substances.
The maximum concentration values of the restricted substances by weight are:
Based on information provided by its suppliers dummy1 designates the product listed below as:
Lead                                                                             0.1%
Mercury                                                                        0.1%
Hexavalent Chromium                                                  0.1%
Polybrominated Biphenyls                                            0.1%
Polybrominated Diphenyl Ethers                                  0.1%
Cadmium                                                                      0.01%

Bis(2-Ethylhexyl) phthalate (DEHP)                             0.1%
Benzyl butyl phthalate (BBP)                                       0.1%
Dibutyl phthalate (DBP)                                               0.1%
Diisobutyl phthalate (DIBP)                                          0.1%

RoHS Phthalates Compliant

Order Code 1190837
Description MULTICOMP PRO SP16/1.2/YELLOW Heat Shrink
Tubing, 2:1, 0.062 ", 1.6 mm, Yellow, 3.9 ft, 1.2 m
Manufacturer / Brand Name   MULTICOMP PRO
Manufacturer Part Number     SP16/1.2/YELLOW
Date                                       09-Aug-2024
Customer                               Bhagyashree Rampur


Product Compliance Senior Manager
Statement on behalf of dummy1
"""

In [13]:
# ========== JSONFORMER GENERATION ==========
from jsonformer.main import Jsonformer
from jsonformer.format import highlight_values

builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=coc,
    #prompt=prompt_use.format(context=sample_context, question=question),
    #prompt=prompt_use.format(context=sample_context2, question=question),
    prompt=prompt_use.format(context=rag_context, question=question),
    max_string_token_length=32   # reduce per-field token count

)

try:
  output = builder()
  highlight_values(output)
except Exception as e:
    print("Error during generation:", e)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{
  coc: {
    Manufacturer Name: "MULTICOMP PRO",
    Document Name: "MULTICOMP PRO",
    Document Date: "2019-08-22",
    Compliance result: True,
    Regulation type: "EN",
    Non compliant substances list: [
      "Mercury"
    ]
  }
}
